In [50]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
import geoplot as gplt
from shapely.geometry import Point
import shapely
import reverse_geocoder as rg

import sklearn as sk
import matplotlib as mpl
import matplotlib.pylab as plt
import matplotlib.font_manager as fm
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set(rc={'figure.figsize':(13.7,10.27)})
sns.set_style("whitegrid")
sns.set_color_codes()

In [23]:
from dask.distributed import Client
import dask.bag as db
import dask.dataframe as dd
import dask.array as da
import dask

from ast import literal_eval
from collections.abc import MutableMapping
from collections import Counter
# import h5py
import io
import os

In [3]:
Client()

Client Scheduler: tcp://127.0.0.1:37689 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 20.70 GB


# Load all datasets

## Respective schemas

In [4]:
users={'userName': str,
      'jobs': object,
      'currentPlace': object,
      'previousPlaces': object,
      'education': object,
      'gPlusUserId': str}
reviews={'rating': float,
       'reviewerName': str,
       'reviewText': str,
       'categories': object,
       'gPlusPlaceId': str,
       'unixReviewTime': object,
       'reviewTime': str,
       'gPlusUserId': str}
places={'name': str,
       'price': str,
       'address': str,
       'hours': object,
       'phone': str,
       'closed': bool,
       'gPlusPlaceId': str,
       'gps': object}

# del users, reviews, places

## Fastload of smaller files

In [5]:
%%time

# 1 min

users_df = dd.read_csv('users/*.csv',
                 blocksize='128MiB',
                 dtype=users,
                 encoding='utf-8',
                 lineterminator='\n').compute()
# reviews_df = dd.read_csv('reviews/*.csv',
reviews_df = dd.read_csv('reviews_restaurants/*.csv',
                 blocksize='32MiB',
                 dtype=reviews,
                 encoding='utf-8',
                 lineterminator='\n').compute()
places_df = dd.read_csv('places/*.csv',
                 blocksize='64MiB',
                 dtype=places,
                 encoding='utf-8',
                 lineterminator='\n').compute()
# del users_df, reviews_df, places_df

CPU times: user 11.9 s, sys: 5.26 s, total: 17.2 s
Wall time: 46.4 s


# Merge datasets

- Users: currentPlace => user_lat, user_long
- Places: gps => places_lat, places_long

In [6]:
print('Users: ', users_df.columns.to_list())
print('Places: ', places_df.columns.to_list())
print('Reviews: ', reviews_df.columns.to_list(), '\n')

# extract userName, currentPlace, gPlusUserId from users
users_df2 = users_df.drop(['jobs', 'previousPlaces', 'education'], axis=1)

# extract name, price, address, gPlusPlaceId, gps
places_df2 = places_df.drop(['hours', 'phone', 'closed'], axis=1)

# extract rating, reviewerName, categories, gPlusPlaceId, gPlusUserId
reviews_df2 = reviews_df.drop(['reviewText', 'unixReviewTime', 'reviewTime'], axis=1)

print('After dropping...')
print('Users: ', users_df2.columns.to_list())
print('Places: ', places_df2.columns.to_list())
print('Reviews: ', reviews_df2.columns.to_list())

# users_df2
# places_df2
reviews_df2.head()
# del users_df2, places_df2, reviews_df2

Users:  ['userName', 'jobs', 'currentPlace', 'previousPlaces', 'education', 'gPlusUserId']
Places:  ['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps']
Reviews:  ['rating', 'reviewerName', 'reviewText', 'categories', 'gPlusPlaceId', 'unixReviewTime', 'reviewTime', 'gPlusUserId'] 

After dropping...
Users:  ['userName', 'currentPlace', 'gPlusUserId']
Places:  ['name', 'price', 'address', 'gPlusPlaceId', 'gps']
Reviews:  ['rating', 'reviewerName', 'categories', 'gPlusPlaceId', 'gPlusUserId']


,rating,reviewerName,categories,gPlusPlaceId,gPlusUserId
0,4.0,william spindler,"['Asian Restaurant', 'Chinese Restaurant']",106591714648856494903,100000032416892623125
1,5.0,william spindler,"['European Restaurant', 'Italian Restaurant', ...",109420033090810328045,100000032416892623125
2,5.0,william spindler,['Barbecue Restaurant'],111623070919810985923,100000032416892623125
3,4.0,william spindler,['Restaurant'],113854191152597312098,100000032416892623125
4,5.0,william spindler,['Mexican Restaurant'],115827996910815192564,100000032416892623125


## Users: currentPlace column => Lat, Long columns

In [7]:
%%time

def string_to_coord(currentPlace):
    coord = None
    lat = long = None
    try:
        coord = eval(currentPlace)[1][1:3]
        lat, long = coord[0]/10000000, coord[1]/10000000
    # bad practice
    except:
        pass
    return (lat, long)

users_df2['Coordinates'] = users_df2['currentPlace'].map(string_to_coord)
users_df2['user_lat'] = users_df2['Coordinates'].apply(lambda x : x[0] if x else None)
users_df2['user_long'] = users_df2['Coordinates'].apply(lambda x : x[1] if x else None)

# rename userName -> reviewerName
users_df2 = users_df2.rename(columns={'userName': 'reviewerName'})
users_df2.head()

CPU times: user 11.7 s, sys: 545 ms, total: 12.3 s
Wall time: 12.2 s


,reviewerName,currentPlace,gPlusUserId,Coordinates,user_lat,user_long
0,an lam,"['Thành phố Hồ Chí Minh, Việt Nam', [[], 10823...",100000010817154263736,"(10.823099, 106.629664)",10.823099,106.629664
1,HALİL TURGUT,"['Adana', [[], 370000000, 353213330, 1]]",100000013500285534661,"(37.0, 35.321333)",37.000000,35.321333
2,森田さとこ,NaN,100000021336848867366,"(None, None)",NaN,NaN
3,amey kore,NaN,100000030557048145331,"(None, None)",NaN,NaN
4,william spindler,NaN,100000032416892623125,"(None, None)",NaN,NaN


#### Make sure the lat, long coordinates are OK

In [ ]:
users_df2.describe()

## Places: Gps column => Lat, Long columns

In [8]:
%%time

def string_to_coord2(gps):
    if type(gps) is not str:
        return None, None
    coord = eval(gps)
    lat, long = coord[0], coord[1]
    if abs(lat) > 90 or abs(long) > 180:
        lat = lat / 1000000
        long = long / 1000000
    return lat, long

places_df2['Coordinates'] = places_df2['gps'].map(string_to_coord2)
places_df2['place_lat'] = places_df2['Coordinates'].apply(lambda x : x[0] if x else None)
places_df2['place_long'] = places_df2['Coordinates'].apply(lambda x : x[1] if x else None)

places_df2 = places_df2.rename(columns={'name': 'placeName'})
places_df2.head()

CPU times: user 20.9 s, sys: 659 ms, total: 21.5 s
Wall time: 21.4 s


,placeName,price,address,gPlusPlaceId,gps,Coordinates,place_lat,place_long
0,Diamond Valley Lake Marina,NaN,"['2615 Angler Ave', 'Hemet, CA 92545']",104699454385822125632,"[33.703804, -117.003209]","(33.703804, -117.003209)",33.703804,-117.003209
1,Blue Ribbon Cleaners,NaN,"['Parole', 'Annapolis, MD']",103054478949000078829,"[38.979759, -76.547538]","(38.979759, -76.547538)",38.979759,-76.547538
2,Portofino,NaN,"['ул. Тутаева, 1', 'Nazran, Ingushetia, Russia...",109810290098030327104,"[43.22776, 44.762726]","(43.22776, 44.762726)",43.227760,44.762726
3,T C's Referee Sports Bar,$$,"['5322 W 26th St', 'Sioux Falls, SD 57106']",100327153115986850675,"[43.529494, -96.792244]","(43.529494, -96.792244)",43.529494,-96.792244
4,Carrefour - Palembang Square,NaN,"['Jl. Angkatan 45', 'Kompleks Palembang Square...",103368487323937936043,"[-2.976256, 104.742662]","(-2.976256, 104.742662)",-2.976256,104.742662


#### Make sure the lat and long coordinates are OK

In [ ]:
places_df2.describe()

## Drop redundant columns and join

In [30]:
%%time
users_df3 = users_df2.drop(['currentPlace', 'Coordinates'], axis=1)
places_df3 = places_df2.drop(['gps', 'Coordinates'], axis=1)

joined_df = reviews_df2.merge(users_df3,
                            how='left',
                            on='gPlusUserId',
                            suffixes=('_left', '_right')).merge(places_df3,
                                                               how='left',
                                                               on='gPlusPlaceId')

del users_df3, places_df3
# del joined_df

joined_df.tail()

CPU times: user 13.4 s, sys: 1.37 s, total: 14.8 s
Wall time: 14.5 s


,rating,reviewerName_left,categories,gPlusPlaceId,gPlusUserId,reviewerName_right,user_lat,user_long,placeName,price,address,place_lat,place_long
4110767,2.0,charles mckinney,"['Hamburger Restaurant', 'Fast Food Restaurant']",116458473784504954830,118446742455312620560,NaN,NaN,NaN,Fatburger,NaN,"['6780 Cherry Ave', 'Long Beach, CA 90805']",33.878048,-118.168365
4110768,2.0,charles mckinney,"['Mexican Restaurant', 'Latin American Restaur...",117332598175065149705,118446742455312620560,NaN,NaN,NaN,Super Mex,NaN,"['5660 Atlantic Ave', 'Long Beach, CA 90805']",33.859250,-118.184753
4110769,4.0,charles mckinney,"['Hot Dog Restaurant', 'Takeout Restaurant', '...",117868066122653879601,118446742455312620560,NaN,NaN,NaN,Wienerschnitzel,$$$,"['1300 E Rosecrans Ave', 'Compton, CA 90221']",33.903287,-118.209676
4110770,1.0,charles mckinney,"['Buffet Restaurant', 'American Restaurant']",117952004983617019485,118446742455312620560,NaN,NaN,NaN,HomeTown Buffet,$$,"['3102 E Imperial Hwy', 'Lynwood, CA 90262']",33.930282,-118.216058
4110771,1.0,charles mckinney,"['Mexican Restaurant', 'Latin American Restaur...",118224418815687780032,118446742455312620560,NaN,NaN,NaN,El Gallo Giro,$$$,"['7148 Pacific Blvd', 'Huntington Park, CA 902...",33.974378,-118.224995


# Analyze and preprocess

In [31]:
joined_df.count()

rating                4110772
reviewerName_left     4110772
categories            4110772
gPlusPlaceId          4110772
gPlusUserId           4110772
reviewerName_right    3321369
user_lat              1022203
user_long             1022203
placeName             4109855
price                 1870612
address               4109855
place_lat             4102647
place_long            4102647
dtype: int64

In [32]:
# drop restaurants without gps coordinates
joined_df = joined_df.dropna(subset=['place_lat', 'place_long'])

# drop reviewerName_right from the users dataset
joined_df = joined_df.drop('reviewerName_right', axis=1)
joined_df = joined_df.rename(columns={'reviewerName_left': 'reviewerName'})

## Let's try focusing on just the United States for now

We need `us_cities.csv` as a custom dataset for reverse_geocoder

Save as `us_cities_rg.csv` for later

In [33]:
%%time
# https://raw.githubusercontent.com/kelvins/US-Cities-Database/main/csv/us_cities.csv
us_cities = dd.read_csv('data/us_cities.csv').compute()

# drop unneeded column
us_cities = us_cities.drop(['ID'], axis=1)

# rename and rearrange into the following format
# lat,lon,name,admin1,admin2,cc
# cc is country_code

# rename
us_cities = us_cities.rename(columns={'LONGITUDE': 'lon',
                                      'LATITUDE': 'lat',
                                      'CITY': 'name',
                                      'STATE_NAME': 'admin1',
                                      'COUNTY': 'admin2'})
# rearrange
us_cities = us_cities[['lat', 'lon', 'name', 'admin1', 'admin2']]
us_cities = us_cities.assign(cc='US')

# save as csv
us_cities.to_csv('data/us_cities_rg.csv', index=False)

del us_cities

CPU times: user 149 ms, sys: 7.23 ms, total: 156 ms
Wall time: 483 ms


In [46]:
geo = rg.RGeocoder(mode=2,
                   verbose=True,
                   stream=io.StringIO(open('data/us_cities_rg.csv',
                                           encoding='utf-8').read()))

contiguous_usa = gpd.read_file(gplt.datasets.get_path('contiguous_usa'))

# check if a gps coord is within the contiguous usa
def within_usa(df):
    result = []
    for _, row in df.iterrows():
        lat = row['place_lat']
        long = row['place_long']
        result.append(any(contiguous_usa.contains(Point(long, lat))))
    return result

# del geo, contiguous_usa

contiguous_usa.head()

,state,adm1_code,population,geometry
0,Minnesota,USA-3514,5303925,"POLYGON ((-89.59941 48.01027, -89.48888 48.013..."
1,Montana,USA-3515,989415,"POLYGON ((-111.19419 44.56116, -111.29155 44.7..."
2,North Dakota,USA-3516,672591,"POLYGON ((-96.60136 46.35136, -96.53891 46.199..."
3,Idaho,USA-3518,1567582,"POLYGON ((-111.04973 44.48816, -111.05025 42.0..."
4,Washington,USA-3519,6724540,"POLYGON ((-116.99807 46.33017, -116.90653 46.1..."


#### Check if the point lies within the contiguous_usa polygon

https://github.com/ResidentMario/geoplot-data

In [48]:
%%time

# filter for reviews within the usa only
# usa_df = joined_df.loc[within_usa]

# runtime 20 min
# usa_df.reset_index(inplace=True, drop=True)

# usa_df.to_csv('data/usa_df.csv', index=False)

CPU times: user 20min 29s, sys: 50.9 s, total: 21min 20s
Wall time: 20min 16s


In [71]:
%%time
usa_meta={'index': int,
         'rating': float,
         'reviewerName': str,
         'categories': object,
         'gPlusPlaceId': str,
         'gPlusUserId': str,
         'user_lat': float,
         'user_long': float,
         'placeName': str,
         'price': str,
         'address': object,
         'place_lat': float,
         'place_long': float,
         'geometry': object}
usa_df = dd.read_csv('data/usa_df.csv', dtype=usa_meta, blocksize='64MiB').compute()
usa_df = usa_df.drop(columns=['index','geometry'])
usa_df.reset_index(inplace=True, drop=True)
usa_df.tail()

CPU times: user 3.4 s, sys: 1.72 s, total: 5.12 s
Wall time: 12.1 s


,rating,reviewerName,categories,gPlusPlaceId,gPlusUserId,user_lat,user_long,placeName,price,address,place_lat,place_long
2087457,2.0,charles mckinney,"['Hamburger Restaurant', 'Fast Food Restaurant']",116458473784504954830,118446742455312620560,NaN,NaN,Fatburger,NaN,"['6780 Cherry Ave', 'Long Beach, CA 90805']",33.878048,-118.168365
2087458,2.0,charles mckinney,"['Mexican Restaurant', 'Latin American Restaur...",117332598175065149705,118446742455312620560,NaN,NaN,Super Mex,NaN,"['5660 Atlantic Ave', 'Long Beach, CA 90805']",33.859250,-118.184753
2087459,4.0,charles mckinney,"['Hot Dog Restaurant', 'Takeout Restaurant', '...",117868066122653879601,118446742455312620560,NaN,NaN,Wienerschnitzel,$$$,"['1300 E Rosecrans Ave', 'Compton, CA 90221']",33.903287,-118.209676
2087460,1.0,charles mckinney,"['Buffet Restaurant', 'American Restaurant']",117952004983617019485,118446742455312620560,NaN,NaN,HomeTown Buffet,$$,"['3102 E Imperial Hwy', 'Lynwood, CA 90262']",33.930282,-118.216058
2087461,1.0,charles mckinney,"['Mexican Restaurant', 'Latin American Restaur...",118224418815687780032,118446742455312620560,NaN,NaN,El Gallo Giro,$$$,"['7148 Pacific Blvd', 'Huntington Park, CA 902...",33.974378,-118.224995


In [72]:
usa_df.count()

rating          2087462
reviewerName    2087462
categories      2087462
gPlusPlaceId    2087462
gPlusUserId     2087462
user_lat         534543
user_long        534543
placeName       2087462
price           1618354
address         2087462
place_lat       2087462
place_long      2087462
dtype: int64

#### Make sure the (lat, long) coordinates are OK

In [73]:
usa_df.describe()

,rating,user_lat,user_long,place_lat,place_long
count,2.087462e+06,534543.000000,534543.000000,2.087462e+06,2.087462e+06
mean,3.970123e+00,37.484354,-90.890976,3.753724e+01,-9.388105e+01
std,1.174481e+00,7.265882,31.383655,5.062171e+00,1.713752e+01
min,0.000000e+00,-90.000000,-175.198242,2.529142e+01,-1.244993e+02
25%,3.000000e+00,33.894680,-112.074037,3.380292e+01,-1.118123e+02
50%,4.000000e+00,38.220610,-89.018722,3.844145e+01,-8.797914e+01
75%,5.000000e+00,41.280411,-79.930922,4.108797e+01,-8.011540e+01
max,5.000000e+00,90.000000,178.000000,4.934520e+01,-6.725381e+01


### State, City, County

In [77]:
lat = usa_df['place_lat']
long = usa_df['place_long']
usa_geo = geo.query(list(zip(lat.to_list(), long.to_list())))
city = []
county = []
state = []
for obj in usa_geo:
    city.append(obj['name'])
    county.append(obj['admin2'])
    state.append(obj['admin1'])
usa_df['state'] = pd.Series(state)
usa_df['county'] = pd.Series(county)
usa_df['city'] = pd.Series(city)
del state, county, city, usa_geo, long, lat

## Which state has the most reviews?

In [78]:
usa_df['state'].value_counts()

California              324612
Texas                   204794
New York                166443
Florida                 140898
Illinois                112808
New Jersey               64180
North Carolina           60420
Georgia                  59008
Oregon                   56686
Washington               56571
Michigan                 55400
Colorado                 52827
Virginia                 52296
Pennsylvania             51731
Ohio                     50841
Arizona                  50503
Massachusetts            48597
Maryland                 34135
Minnesota                32317
Wisconsin                32094
Missouri                 31406
Tennessee                30111
Indiana                  28876
South Carolina           27491
Utah                     25019
Nevada                   21596
Louisiana                20145
Oklahoma                 17425
Connecticut              16561
Alabama                  16317
Kansas                   15787
Kentucky                 15375
Iowa    

# Finish the joined EDA

# Ideas

**IDEA**: Merge restaurant types

e.g.
- Ice cream shop, Bakery, Dessert Shop => Dessert Shop
- Thai, South Asian, Southeast Asian, Japanese, Chinese => Asian
- etc...

**IDEA**: Make recommendations by COUNTY

e.g. Santa clara county, san mateo county, etc...
1. Filter for reviews in US
2. Find state with the most reviews ( see places_eda.ipynb for a cartogram )
3. Split the dataset by County
4. Create similarity matrix for each county
5. ...